In [1]:
# %pip install opencv-python
%pip install -U tensorflow-addons -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
import numpy as np 
import pandas as pd 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as k

tf.keras.backend.clear_session()
k.clear_session()

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
feature_desc = {
    'VV': tf.io.FixedLenFeature([], tf.string),
    'VV_1': tf.io.FixedLenFeature([], tf.string),
    'VH': tf.io.FixedLenFeature([], tf.string),
    'VH_1': tf.io.FixedLenFeature([], tf.string),
    'lwe_thickness_jpl': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.float32)
}

In [10]:
def parse_tfrecord(example):
    
    example = tf.io.parse_single_example(example, feature_desc)
    bands = ['VV', 'VV_1', 'VH', 'VH_1', 'lwe_thickness_jpl']
    decoded_bands = [tf.io.decode_raw(example[band], tf.float32) for band in bands]
    reshaped_bands = [tf.reshape(band, (256, 256)) for band in decoded_bands]
    
    # Cast the reshaped bands to float32 if needed
    reshaped_bands = [tf.cast(band, tf.float32) for band in reshaped_bands]
    
    bands_normalized = [tf.divide(band, 255.0) for band in reshaped_bands]
    
    # Use tf.stack instead of tf.concat
    image = tf.stack(bands_normalized, axis=-1)

    label = example['label']
    return image, label

def load_dataset(pattern):
    
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob)
    dataset = dataset.map(parse_tfrecord)
    
    return dataset

def get_dataset():
    
    glob = 'E:\Ground-Water-Monitoring\Data\_tf_record_files\grace_sentinel' + '*'
    dataset = load_dataset(glob)
    
    return dataset

In [ ]:
training = get_dataset()

In [ ]:
train_data = training.take(20000).batch(32).repeat(30)
validation_data = training.skip(20000).batch(1).repeat(30)

In [ ]:
next(iter(train_data))

In [6]:
from transformers import ViTFeatureExtractor, TFViTModel
from datasets import load_dataset
from tensorflow.keras import layers
from tensorflow.keras import losses

import tensorflow as tf
import datetime
import tensorflow_addons as tfa

def mlp(x, hidden_units, dropout_rate):
    
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu) (x)
        x = layers.Dropout(dropout_rate)(x)
    return x   

c:\Users\hp\.conda\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\.conda\envs\textS\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [7]:
class Patches(layers.Layer):
    
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
        
    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = 'VALID'
        )
        
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        
        return patches

In [8]:
class PatchEncoder(layers.Layer):
    def __init__(self, n_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.n_patches = n_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_encoding = layers.Embedding(
            input_dim=n_patches, output_dim=projection_dim
        )
        
    def call(self, patch):
        positions = tf.range(start=0, limit=self.n_patches, delta=1)
        encoded = self.projection(patch) + self.position_encoding(positions)
        
        return encoded

In [9]:
def create_vit_regression():
    
    inputs = layers.Input(shape=input_shape)
    
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(n_patches, projection_dim)(patches)
    
    for ind in range(transformer_layers):
        
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        attention_output = layers.MultiHeadAttention(
            num_heads=n_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        encoded_patches = layers.Add()([x3, x2])
        
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    
    ground_water = layers.Dense(1)(features)
    model = keras.Model(inputs=inputs, outputs=ground_water)
    
    return model

In [ ]:
# configuring the hyper-parameters.

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
epochs = 100
image_size = 256
patch_size = 16
n_patches = (image_size // patch_size) ** 2
projection_dim = 64
n_heads = 4
input_shape = (256, 256, 4)
transformer_units = [
    projection_dim * 2,
    projection_dim,
]

transformer_layers = 8
mlp_head_units = [2048, 1024] # size of the dense layers of the final classifier.

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

def mae(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))

def train_model(model):
    
    optimizer = tfa.optimizers.AdamW(
        learning_rate = learning_rate, weight_decay = weight_decay
    )
    
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=[
            rmse,
            mae
        ]
    )
    
    checkpoint_path = "checkpoint/ViT/"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor='val_rmse',
        save_best_only=True,
        save_weights_only=True
    )
    model = get_model()

    log_dir = "logs/fit/ViT-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq=100)

    
    history = model.fit(
        x=train_data,
        epochs=30,
        steps_per_epoch=20000/32,
        validation_data=validation_data,
        validation_steps=4652,
        callbacks=[tensorboard_callback, checkpoint_callback]
        )
    
    return history

vit_regressor = create_vit_regression()
history = train_model(vit_regressor)